# Get the Data

## Download the Data

In [29]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

In [30]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


## Create a Test Set

In [31]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd

# to make this notebook's output identical at every run
np.random.seed(42)

housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])


split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

strat_test_set["income_cat"].value_counts() / len(strat_test_set)


income_cat
3    0.350533
2    0.318798
4    0.176357
5    0.114341
1    0.039971
Name: count, dtype: float64

# Prepare the Data for Machine Learning Algorithms

In [32]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)

## Custom Transformers

In [33]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat,rooms_per_household,population_per_household
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND,2,5.485836,3.168555
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN,5,6.927083,2.623698
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.875,INLAND,2,5.393333,2.223333
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN,2,3.886128,1.859213
20496,-118.7,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN,3,6.096552,3.167241


In [34]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

# print out shape of the resulting array
print(housing_prepared.shape)

(16512, 17)


# Select and Train a Model

## Training and Evaluating on the Training Set

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

# lin_reg = LinearRegression()
# lin_reg.fit(housing_prepared, housing_labels)

# lin_predictions = lin_reg.predict(housing_prepared)
# lin_mse = mean_squared_error(housing_labels, lin_predictions)
# lin_rmse = np.sqrt(lin_mse)

# print("lin_rmse:", lin_rmse)

# lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
#                          scoring="neg_mean_squared_error", cv=10)
# lin_rmse_scores = np.sqrt(-lin_scores)
# display_scores(lin_rmse_scores)


lin_rmse: 68376.51254853733
lin_mae: 49507.34233776102
Scores: [71523.78333874 64044.46774989 67454.97869698 68514.10137273
 66303.62531226 72166.63405138 74464.08841381 68570.11804395
 66063.64175868 69870.86192291]
Mean: 68897.63006613276
Standard deviation: 3002.746127534865


### Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# tree_reg = DecisionTreeRegressor(random_state=42)
# tree_reg.fit(housing_prepared, housing_labels)
# tree_predictions = tree_reg.predict(housing_prepared)
# tree_mse = mean_squared_error(housing_labels, tree_predictions)
# tree_rmse = np.sqrt(tree_mse)
# print("tree_rmse:", tree_rmse)

# tree_scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
#                          scoring="neg_mean_squared_error", cv=10)
# tree_rmse_scores = np.sqrt(-tree_scores)
# display_scores(tree_rmse_scores)


tree_rmse: 0.0
Scores: [72237.45096271 69208.53094621 69286.11573206 70612.13634405
 70577.30202088 76840.97092314 71823.83279699 72710.22512908
 67780.77127665 71678.17802175]
Mean: 71275.55141535088
Standard deviation: 2362.02756776275


### Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
# forest_reg.fit(housing_prepared, housing_labels)
# forest_predictions = forest_reg.predict(housing_prepared)
# forest_mse = mean_squared_error(housing_labels, forest_predictions)
# forest_rmse = np.sqrt(forest_mse)
# print("forest_rmse:", forest_rmse)

# forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
#                          scoring="neg_mean_squared_error", cv=10)
# forest_rmse_scores = np.sqrt(-forest_scores)
# display_scores(forest_rmse_scores)

forest_rmse: 18675.224916252282
Scores: [51553.65292335 48797.89565614 47005.23947642 52046.73567245
 47700.78025873 51824.08544879 52582.59165129 49949.79025967
 48680.25622229 54019.67674791]
Mean: 50416.070431704204
Standard deviation: 2201.612779754884


### Support Vector Machine Regression

In [ ]:
from sklearn.svm import SVR
# svm_reg = SVR(kernel="linear")
# svm_reg.fit(housing_prepared, housing_labels)
# svm_predictions = svm_reg.predict(housing_prepared)
# svm_mse = mean_squared_error(housing_labels, svm_predictions)
# svm_rmse = np.sqrt(svm_mse)
# print("svm_rmse:", svm_rmse)

# svm_scores = cross_val_score(svm_reg, housing_prepared, housing_labels,
#                          scoring="neg_mean_squared_error", cv=10)
# svm_rmse_scores = np.sqrt(-svm_scores)
# display_scores(svm_rmse_scores)


svm_rmse: 106873.526604078
Scores: [106846.91418486 108840.55721006 103122.02433423 109649.5401006
 103796.57205905 112219.65143015 109351.47147536 107877.83853403
 109166.01514235 108053.90140047]
Mean: 107892.44858711655
Standard deviation: 2590.6419916301356


## Fine-Tune Your Model

### Grid Search

#### Linear Regression

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [38]:
param_grid = [
        {'fit_intercept': [True, False], 'positive': [True, False]},
    ]
lin_reg = LinearRegression()
lin_grid_search = GridSearchCV(lin_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
lin_grid_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


,estimator,LinearRegression()
,param_grid,"[{'fit_intercept': [True, False], 'positive': [True, False]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,fit_intercept,True


In [39]:
# This is basically unnecessary since there are only 4 combinations, but for completeness:
print("\nBest hyperparameters:")
print(lin_grid_search.best_params_)


Best hyperparameters:
{'fit_intercept': True, 'positive': False}


#### Decision Tree Regression

In [55]:
param_grid = [
        {'max_features': [3, 6, 10, 13], 'max_depth': [10, 20, 30, 40, 50],
         'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    ]
tree_reg = DecisionTreeRegressor(random_state=42)
tree_grid_search = GridSearchCV(tree_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=2, n_jobs=-1)
tree_grid_search.fit(housing_prepared, housing_labels)


Fitting 5 folds for each of 180 candidates, totalling 900 fits


,estimator,DecisionTreeR...ndom_state=42)
,param_grid,"[{'max_depth': [10, 20, ...], 'max_features': [3, 6, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 5, ...]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,criterion,'squared_error'


In [56]:
param_distribs = {
        'max_features': randint(low=1, high=17),
        'max_depth': randint(low=10, high=100),
        'min_samples_split': randint(low=2, high=20),
        'min_samples_leaf': randint(low=1, high=10)
    }
tree_reg = DecisionTreeRegressor(random_state=42)
tree_rnd_search = RandomizedSearchCV(tree_reg, param_distribs, n_iter=50, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=2, n_jobs=-1)
tree_rnd_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


,estimator,DecisionTreeR...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....0023A4D3FE2F0>, 'max_features': <scipy.stats....0023A4D3FF760>, 'min_samples_leaf': <scipy.stats....0023A4D3C5D80>, 'min_samples_split': <scipy.stats....0023A5060F970>}"
,n_iter,50
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [ ]:
cvres = tree_rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Randomized:", np.sqrt(-mean_score), params)

cvres = tree_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Grid:", np.sqrt(-mean_score), params)

print("\nBest hyperparameters:")
print("Grid:")
print(tree_rnd_search.best_params_)
print("Randomized:")
print(tree_grid_search.best_params_)

Randomized: 63847.365976932924 {'max_depth': 12, 'max_features': 6, 'min_samples_leaf': 5, 'min_samples_split': 3}
Randomized: 62412.93566293121 {'max_depth': 97, 'max_features': 12, 'min_samples_leaf': 6, 'min_samples_split': 3}
Randomized: 63654.247062686256 {'max_depth': 73, 'max_features': 12, 'min_samples_leaf': 5, 'min_samples_split': 2}
Randomized: 62388.96950243967 {'max_depth': 85, 'max_features': 10, 'min_samples_leaf': 6, 'min_samples_split': 13}
Randomized: 74586.93020982658 {'max_depth': 98, 'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 13}
Randomized: 62825.85075529686 {'max_depth': 64, 'max_features': 4, 'min_samples_leaf': 9, 'min_samples_split': 4}
Randomized: 64795.115781438006 {'max_depth': 60, 'max_features': 7, 'min_samples_leaf': 5, 'min_samples_split': 10}
Randomized: 68664.08423475416 {'max_depth': 48, 'max_features': 2, 'min_samples_leaf': 4, 'min_samples_split': 15}
Randomized: 64046.33429117243 {'max_depth': 18, 'max_features': 10, 'min_sampl

#### Random Forest Regression

In [52]:
param_grid = [
    {'n_estimators': [1, 10, 100, 300], 'max_features': [2, 6, 10, 13]},
    {'bootstrap': [False], 'n_estimators': [1, 10], 'max_features': [2, 4, 6]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
forest_grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=2, n_jobs=-1)
forest_grid_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 22 candidates, totalling 110 fits


,estimator,RandomForestR...ndom_state=42)
,param_grid,"[{'max_features': [2, 6, ...], 'n_estimators': [1, 10, ...]}, {'bootstrap': [False], 'max_features': [2, 4, ...], 'n_estimators': [1, 10]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,n_estimators,300


In [54]:
param_distribs = {
        'n_estimators': randint(low=1, high=500),
        'max_features': randint(low=1, high=17),
    }

forest_reg = RandomForestRegressor(random_state=42)
forest_rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42, verbose=2, n_jobs=-1)
forest_rnd_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


,estimator,RandomForestR...ndom_state=42)
,param_distributions,"{'max_features': <scipy.stats....0023A40C3C8E0>, 'n_estimators': <scipy.stats....0023A4D3FEFE0>}"
,n_iter,20
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [65]:
cvres = forest_rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Randomized:", np.sqrt(-mean_score), params)

cvres = forest_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Grid:", np.sqrt(-mean_score), params)

print("\nBest hyperparameters:")
print("Grid:")
print(forest_grid_search.best_params_)
print("Randomized:")
print(forest_rnd_search.best_params_)

Randomized: 49679.385063605514 {'max_features': 7, 'n_estimators': 436}
Randomized: 50164.249988275566 {'max_features': 13, 'n_estimators': 271}
Randomized: 50221.78165952025 {'max_features': 11, 'n_estimators': 72}
Randomized: 51170.41799916229 {'max_features': 13, 'n_estimators': 21}
Randomized: 49958.42482333546 {'max_features': 7, 'n_estimators': 122}
Randomized: 50819.31614853545 {'max_features': 3, 'n_estimators': 215}
Randomized: 49960.2728415402 {'max_features': 11, 'n_estimators': 459}
Randomized: 49814.23817320756 {'max_features': 8, 'n_estimators': 373}
Randomized: 50225.060329059015 {'max_features': 4, 'n_estimators': 360}
Randomized: 49940.32510872611 {'max_features': 8, 'n_estimators': 131}
Randomized: 49836.64564347814 {'max_features': 6, 'n_estimators': 309}
Randomized: 51877.015575827936 {'max_features': 2, 'n_estimators': 344}
Randomized: 50038.361361457704 {'max_features': 12, 'n_estimators': 414}
Randomized: 49862.63570037307 {'max_features': 6, 'n_estimators': 386}

#### Support Vector Machine Regression

In [57]:
param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
        {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svm_reg = SVR()
svm_grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
svm_grid_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


,estimator,SVR()
,param_grid,"[{'C': [10.0, 30.0, ...], 'kernel': ['linear']}, {'C': [1.0, 3.0, ...], 'gamma': [0.01, 0.03, ...], 'kernel': ['rbf']}]"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'linear'


In [58]:
from scipy.stats import expon, reciprocal

# Note: gamma is ignored when kernel is "linear"
param_distribs = {
        'kernel': ['linear', 'rbf'],
        'C': reciprocal(20, 200000),
        'gamma': expon(scale=1.0),
    }

svm_reg = SVR()
svm_rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='neg_mean_squared_error',
                                verbose=2, random_state=42, n_jobs=-1)
svm_rnd_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


,estimator,SVR()
,param_distributions,"{'C': <scipy.stats....0023A506DFEB0>, 'gamma': <scipy.stats....0023A4D3FFD60>, 'kernel': ['linear', 'rbf']}"
,n_iter,50
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [66]:
cvres = svm_rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Randomized:", np.sqrt(-mean_score), params)

cvres = svm_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Grid:", np.sqrt(-mean_score), params)

print("\nBest hyperparameters:")
print("Grid:")
print(svm_grid_search.best_params_)
print("Randomized:")
print(svm_rnd_search.best_params_)

Randomized: 70300.98256946394 {'C': np.float64(629.7823295913721), 'gamma': np.float64(3.010121430917521), 'kernel': 'linear'}
Randomized: 67044.35375848124 {'C': np.float64(26290.20646430022), 'gamma': np.float64(0.9084469696321253), 'kernel': 'rbf'}
Randomized: 96909.67164587777 {'C': np.float64(84.14107900575871), 'gamma': np.float64(0.059838768608680676), 'kernel': 'rbf'}
Randomized: 70367.72057197081 {'C': np.float64(432.37884813148844), 'gamma': np.float64(0.15416196746656105), 'kernel': 'linear'}
Randomized: 118863.67394520435 {'C': np.float64(24.175082946113903), 'gamma': np.float64(3.503557475158312), 'kernel': 'rbf'}
Randomized: 69667.46259483059 {'C': np.float64(113564.03940586244), 'gamma': np.float64(0.0007790692366582295), 'kernel': 'rbf'}
Randomized: 108097.17539031299 {'C': np.float64(108.30488238805071), 'gamma': np.float64(0.3627537294604771), 'kernel': 'rbf'}
Randomized: 75845.61956696333 {'C': np.float64(21.34495367264743), 'gamma': np.float64(0.023332523598323388),